<a href="https://colab.research.google.com/github/MathurUtkarsh/Amazon-Scraping-/blob/main/Python_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  bs4
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Define the base URL and headers
base_url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&page="
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
}

# Define an empty list to store the scraped product data
data = []

# Loop over the first 20 pages of the product listing
for page_num in range(1, 21):
    # Get the URL for the current page
    url = base_url + str(page_num)
    
    # Get the page content
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    
    # Find all the product containers on the page
    product_containers = soup.find_all("div", class_="s-result-item")
    
    # Loop over each product container
    for product in product_containers:
        try:
            # Extract the product information from the container
            product_url = product.find("a", class_="a-link-normal").get("href")
            product_name = product.find("span", class_="a-size-medium a-color-base a-text-normal").text.strip()
            product_price = product.find("span", class_="a-offscreen").text.strip()
            product_rating = product.find("span", class_="a-icon-alt").text.strip()
            product_reviews = product.find("span", class_="a-size-base s-underline-text").text.strip()
            
            # Get the product page content
            product_page = requests.get(product_url, headers=headers)
            product_soup = BeautifulSoup(product_page.content, "html.parser")
            
            # Extract the additional product information
            product_description = product_soup.find("div", id="feature-bullets").text.strip()
            product_asin = product_soup.find("input", id="ASIN").get("value")
            product_manufacturer = product_soup.find("div", id="bylineInfo").text.strip()
            
            # Store the product information in the data list
            data.append({
                "Product URL": product_url,
                "Product Name": product_name,
                "Product Price": product_price,
                "Rating": product_rating,
                "Number of Reviews": product_reviews,
                "Description": product_description,
                "ASIN": product_asin,
                "Manufacturer": product_manufacturer
            })
        except:
            # Skip any product containers that don't have all the required information
            continue

# Convert the data list to
df = pd.DataFrame(data)


In [ ]:
df.to_csv("product_data.csv", index=False)